#TITRE

In [61]:
Left = True
Right = False

In [132]:
import math

class AbstractRule:
    
    def _set_grammar(self,gram):
        self._grammar = gram
        
    def valuation(self):
        raise NotImplementedError
        
    def count(self):
        raise NotImplementedError
        
    def listR(self,i):
        raise NotImplementedError
        
    def unrank(self,n,i):
        raise NotImplementedError

    def rank(self,obj):
        raise NotImplementedError
        
#Exception triggered when a rule uses a literal not in the grammar
class UnknownLiteralError(Exception):
    
    def __init__(self,literal):
        self.literal = literal
        
#Exception triggered when an unrank goes wrong
class ValueError(Exception):
    
    def __init__(self, c,i):
        self.count = c
        self.rank = i
        
#Exception triggered when a rank goes wrong
class ObjectError(Exception):
    pass

#This is not supposed to happen : during unrank
#no elements have been found with the specified rank,
#but we check for this before, so this is not supposed to happen
class NoBoxFound (Exception) :
    pass

class ConstructorRule(AbstractRule):
    
    def __init__(self, *args):
        self._parameters = args
        #print (self._parameters[0])
        #print (self._parameters[1])
        self._valuation = math.inf
        self._revCons = self._parameters[3]
        self._constructor = self._parameters[2]
        self._counts = {}

    def valuation(self):
        return self._valuation
    
    #The Grammar is supposed non ambiguous
    #check if all literals used in the rule are parts of the grammar
    def _verif_rule(self):
        try:
            self._grammar[self._parameters[0]]
        except KeyError:
            raise UnknownLiteralError(self._parameters[0])
        try:
            self._grammar[self._parameters[1]]
        except KeyError:
            raise UnknownLiteralError(self._parameters[1])

    #Return true if there was no update
    def _update_valuation(self):
            #print("Valuation : Constructor")
            #print("old :"+str(self._valuation))
            self._old_val = self._valuation
            self._valuation = self._calc_valuation()
            #print("new :"+str(self._valuation))
            #print(self._old_val == self._valuation)
            return (self._old_val == self._valuation)
    
    def cleanCounts(self):
        self._counts = {}

        
class UnionRule(ConstructorRule):
    
    def __init__(self,fst,snd,cons,rev):
        #print ("union")
        super().__init__(fst,snd,cons,rev)
        
    def _calc_valuation(self):
        #print ("union")
        return min(self._grammar[self._parameters[0]].valuation(),
                   self._grammar[self._parameters[1]].valuation())
    
    def count(self, i):
        #print (self._parameters[0])
        #print (self._parameters[1])
        #print (self._grammar[self._parameters[0]].count(i))
        #print (self._grammar[self._parameters[1]].count(i))
        try:
            return self._counts[i]
        except KeyError:
            c = self._grammar[self._parameters[0]].count(i) + self._grammar[self._parameters[1]].count(i)
            self._counts[i] = c
            return c
            
    def listR(self, i):
        return [self._constructor(Left,x) for x in self._grammar[self._parameters[0]].listR(i)]\
                + [self._constructor(Right,x) for x in self._grammar[self._parameters[1]].listR(i)]
    
    def unrank(self,n,i):
        #print("unrank union "+self._parameters[0]+" "+self._parameters[1])
        #print("n: "+str(n))
        if(i>=self.count(n)):
            raise ValueError(self.count(n),i)
        r0 = self._grammar[self._parameters[0]]
        r1 = self._grammar[self._parameters[1]]
        c0 = r0.count(n)
        if i < c0:
            #print("chosen : "+self._parameters[0])
            return self._constructor(Left,r0.unrank(n,i))
        else:
            #print("chosen : "+self._parameters[0])
            return self._constructor(Right,r1.unrank(n,i-c0))
        
    def rank(self, obj):
        v,n,newObj = self._revCons(obj)
        r0 = self._grammar[self._parameters[0]]
        c0 = r0.count(n)
        if(v):
            #print("Left side of union rank")
            #print("newObj : "+str(newObj))
            return r0.rank(newObj)
        else:
            print("Right side of union rank")
            print("newObj : "+str(newObj))
            print("c0:",str(c0))
            return c0+self._grammar[self._parameters[1]].rank(newObj)


class ProductRule(ConstructorRule):
    
    def __init__(self,fst,snd,cons,rev):
        #print("product")
        super().__init__(fst,snd,cons,rev)
    def _calc_valuation(self):
        #print("product")
        return (self._grammar[self._parameters[0]].valuation() +\
               self._grammar[self._parameters[1]].valuation())
    
    def count(self, i):
        #print("count : product")
        try:
            return self._counts[i]
        except KeyError:
            memi = i
            res = 0
            valN1 = self._grammar[self._parameters[0]].valuation()
            valN2 = self._grammar[self._parameters[1]].valuation()
            #print (self._parameters[0])
            #print (self._parameters[1])
            #print("i :" + str(i))
            #print("valN1 :" + str(valN1))
            #print("valN2 :" + str(valN2))
            for k in range (valN1, i-valN2+1):
                #print("k : "+str(k))
                #print("l : "+str(i-k))
                res += self._grammar[self._parameters[0]].count(k) * self._grammar[self._parameters[1]].count(i-k)
                #print("res : "+str(res))
            self._counts[memi] = res
            return res
    
    def listR(self, i):
        #print("listR : product")
        res = []
        valN1 = self._grammar[self._parameters[0]].valuation()
        valN2 = self._grammar[self._parameters[1]].valuation()
        #print (self._parameters[0])
        #print (self._parameters[1])
        #print("i :" + str(i))
        #print("valN1 :" + str(valN1))
        #print("valN2 :" + str(valN2))
        for k in range (valN1, i-valN2+1):
            #print("k : "+str(k))
            #print("l : "+str(i-k))
            list0 = self._grammar[self._parameters[0]].listR(k)
            list1 = self._grammar[self._parameters[1]].listR(i-k)
            res += [self._constructor(x,y) for x in list0 for y in list1]
            
            #print("res : "+str(res))
        return res

    def unrank(self,n,i):
        #print("unrank product "+self._parameters[0]+" "+self._parameters[1])
        if(i>=self.count(n)):
            raise ValueError(self.count(n),i)
        r0 = self._grammar[self._parameters[0]] #a
        r1 = self._grammar[self._parameters[1]] #b
        valN0 = r0.valuation()
        valN1 = r1.valuation()
        k = valN0
        while True:
            if k > n-valN1 :
                raise NoBoxFound
            c0 = r0.count(k)
            c1 = r1.count(n-k)
            #On cherche la bonne boite
            if i >= c0*c1 :
                i = i - c0*c1
                k+=1
            #On a trouvé
            else :
                q = i // c1
                r = i % c1
                #print("k: "+str(k))
                #print("l: "+str(n-k))
                return self._constructor(r0.unrank(k,q),r1.unrank(n-k,r))
                
    def rank(self, obj):
        k,l,left,right = self._revCons(obj)
        print("obj= "+str(obj))
        print("k="+str(k)+" l="+str(l))
        print(left)
        print(right)
        r0 = self._grammar[self._parameters[0]] #a
        r1 = self._grammar[self._parameters[1]] #b
        valN0 = r0.valuation()
        acc = 0
        for i in range(valN0,k):
            #print("acc : " + str(acc) + " i : " + str(i))
            c0 = r0.count(i)
            c1 = r1.count(k+l-i)
            acc+= c0 * c1
        #print("acc : " + str(acc) + " k : " + str(k) + " l : " + str(l))
        return acc + r0.rank(left) * r1.count(l) + r1.rank(right)
    
    
class ConstantRule(AbstractRule):
    
    def __init__(self,obj):
        self._object = obj
        
    #We chose to put _update_valuation here too
    #to avoid testing the presence of the function
    #every time we want to call it on a rule
    def _update_valuation(self):
        return True

    
class EpsilonRule(ConstantRule):
    
    def __init__(self,obj):
        super().__init__(obj)
    
    def valuation(self):
        return 0
    
    def count(self, i):
        if i != 0:
            return 0
        else :
            return 1
        
    def listR(self,i):
        if i == 0:
            return [self._object]
        else:
            return []
        
    def unrank(self,n,i):
        #print("unrank epsilon "+self._object)
        if i == 0 :
            return self._object
        else:
            raise ValueError
            
    def rank(self,obj):
        if obj == self._object :
            return 0
        else:
            raise ObjectError
    
class SingletonRule(ConstantRule):
    
    def __init__(self,obj):
        super().__init__(obj)
        
    def valuation(self):
        return 1
    
    def count(self, i):
        if i != 1:
            return 0
        else :
            return 1
        
    def listR(self,i):
        if i == 1:
            return [self._object]
        else:
            return []
        
    def unrank(self,n,i):
        #print("unrank singleton "+self._object)
        if i==0:
            return self._object
        else:
            raise ValueError
            
    def rank(self,obj):
        if obj == self._object :
            return 0
        else :
            raise ObjectError

#Triggered when a non terminal does not generate anything
class CircularGrammarError(Exception):
    
    def __init__(self,grammar,name):
        self.name = name
        self.grammar = grammar
        
    
def init_grammar(gram):
    #set the grammar for all rules
    for rule in gram.values() :
        rule._set_grammar(gram)
        #verifies every non-constant rule (check if all literal used in the rules are in the grammar)
        if isinstance(rule, ConstructorRule):
            rule._verif_rule()

    #As long as there's a change we update again
    #(we chose to make _update_valuation available
    #to any rule)
    while not all(rule._update_valuation() for rule in gram.values()):
        pass
    for name,rule in gram.items() :
        if rule.valuation() == math.inf :
            raise CircularGrammarError(gram,name)

**__Specification__**

* count = len(list) <br>
* unrank(i) = list[i] <br>
* rank(unrank(i)) = i <br>
* unrank(rank(obj)) = obj <br>
* if valuation = i  => count(i-1) = 0 <br>



In [133]:
#Timeout for the tests
#this cell  works only for UNIX OS
"""
import signal

class TimeoutError(Exception):
    pass

def timeout(signum,frame):
    raise TimeoutError
    
signal.signal(signal.SIGALRM, timeout)

#TODOCorrect
#Testing all the list with count (in a limited time stamp)

@param grammar : the grammar which is tested
@param base : the litteral that generates elements of the grammar
@param ite_range : useless here, only for thee windows spec compatibility

def test_Count_List_Until_Timeout (grammar, base, ite_range):
    
    i = 0
    try:
        while True:
            signal.alarm(1)
            l = grammar[base].listR(i)
            c = grammar[base].count(i)
            signal.alarm(0)
            assert (c == len(l))
            #Check if there is no multiple appearance of the same value
            assert (len(l) == len(set(l)))
            #print(i)
            i+=1
    except TimeoutError :
        print("the greatest test was for i = " + str(i-1))

"""
pass

In [117]:
#Replacement of the Unix specific function up above
def test_Count_List_Until_Timeout (grammar, base, ite_range):
    for i in range(0,ite_range):
        l = grammar[base].listR(i)
        c = grammar[base].count(i)
        assert (c == len(l))
        #Check if there is no multiple appearance of the same value
        #print("i="+str(i))
        #print("l:"+str(l))
        assert (len(l) == len(set(l)))


In [118]:
#Common imports to all the tests
import operator
    
class EndOfObjError (Exception):
    pass
    
def test_unrank(grammar, base, ite_range):
    
    for n in range(1,ite_range):
        #We suppose that listR works normally
        #print("n="+str(n))
        l = grammar[base].listR(n)
        
        try:
            grammar[base].unrank(n,len(l))
            assert(False)
        except ValueError :
            pass
            
        for i in range(0,len(l)):
            try :
                #print("i="+str(i))
                assert(grammar[base].unrank(n,i) == l[i])
            except Exception as e:
                print("(n,i) :")
                print((n,i))
                print("unrank result :")
                print(grammar[base].unrank(n,i))
                print("corresponding element, then entire list :")
                print(l[i])
                print(l)
                raise e
                
    print("Test unrank ok")
    
def test_rank(grammar, base, ite_range):
    
    for n in range(1,ite_range):
        #We suppose that listR works normally
        #print("__n__="+str(n))
        l = grammar[base].listR(n)
        """ 
        try:
            grammar[base].rank("1asadsf jnqzfnbs")
            assert(False)
        except ValueError :
            pass
        """
        for i in range(0,len(l)):
            try :
                #print("_i_="+str(i))
                assert(grammar[base].rank(l[i]) == i)
            except Exception as e:
                print("(n,i) :")
                print((n,i))
                print("rank result :")
                print(grammar[base].rank(l[i]))
                print("corresponding element :")
                print(i)
                print("entire list, and matched element")
                print(l)
                print(l[i])
                raise e
                
    print("Test rank ok")

In [134]:
def cleanString (s):
    return s.replace("[","").replace("]","").replace("|","").replace(",","").replace("L","").replace("R","")

def cleanList (l):
    return [cleanString(i) for i in l]

def consUnion(v,obj):
    #print("call to consTree")
    if v:
        return "L|"+obj
    else:
        #print("Right side of consTree")
        return "R|"+obj
    
def revUnion(obj):
    #print("call to revTree")
    n = len(cleanString(obj))
    if obj[0] == "L":
        #print("Left side of revTree")
        return Left,n,obj[2:]
    else:
        #print("Right side of revTree")
        return Right,n,obj[2:]
        
def consProd(a,b):
    #print("call to consProd")
    return "["+a+","+b+"]"
    
def revProd(obj) :
    #print("call to revProd")
    parCount = 0
    i=1
    temp = 0
    k = 0
    l = 0
    while i < len(obj)-2 :
        c = obj[i]
        if c == "[":
            parCount += 1
            temp += 1
            l += 1
        elif c == "]" :
            parCount -= 1
        elif c == "," and parCount == 0:
            left = obj[1:i]
            right = obj[i+1:len(obj)-1]
            k = temp
            l = 0
        i+=1
    return k+1,l+1,left,right


    

__Grammar for Binary Trees :__

T --> N | leaf <br>
N --> TT

In [120]:

#Grammar definition for binary trees
def consTree(v,obj):
    #print("call to consTree")
    if v:
        return obj
    else:
        #print("Right side of consTree")
        return obj
    
def revTree(obj):
    #print("call to revTree")
    n = len(cleanString(obj))//4
    if obj[0] == "(":
        #print("Left side of revTree")
        return Left,n,obj
    else:
        #print("Right side of revTree")
        return Right,n,obj
        
def consNode(a,b):
    #print("call to revTree")
    return "("+a+","+b+")"
    
def revNode(obj) :
    #print("call to revNode")
    parCount = 0
    i=1
    temp = 0
    k = 0
    l = 0
    while i < len(obj)-2 :
        c = obj[i]
        if c == "(":
            parCount += 1
            temp += 1
            l += 1
        elif c == ")" :
            parCount -= 1
        elif c == "," and parCount == 0:
            left = obj[1:i]
            right = obj[i+1:len(obj)-1]
            k = temp
            l = 0
        i+=1
    return k+1,l+1,left,right

        
        
treeGram = {"Tree" : UnionRule("Node","Leaf",consTree,revTree),
            "Node" : ProductRule("Tree","Tree",consNode,revNode),
            "Leaf" : SingletonRule("Leaf")}

init_grammar(treeGram)

In [121]:
#tests for the binary trees grammar

#valuation

assert (treeGram["Tree"].valuation() == 1)
assert (treeGram["Node"].valuation() == 2)
assert (treeGram["Leaf"].valuation() == 1)

#count
assert (treeGram["Tree"].count(0) == 0)
assert (treeGram["Tree"].count(1) == 1)
assert (treeGram["Tree"].count(2) == 1)
assert (treeGram["Tree"].count(3) == 2)
assert (treeGram["Tree"].count(4) == 5)
assert (treeGram["Tree"].count(5) == 14)
assert (treeGram["Tree"].count(6) == 42)
assert (treeGram["Tree"].count(7) == 132)
assert (treeGram["Tree"].count(8) == 429)
assert (treeGram["Tree"].count(9) == 1430)


#listR
assert (treeGram["Tree"].listR(0) == [])
#print(treeGram["Tree"].listR(2))
assert (treeGram["Tree"].listR(1) == ["Leaf"])
assert (set(treeGram["Tree"].listR(2)) == set(["(Leaf,Leaf)"]))
assert (set(treeGram["Tree"].listR(3)) == set(["(Leaf,(Leaf,Leaf))",
                                     "((Leaf,Leaf),Leaf)"]))
assert (set(treeGram["Tree"].listR(4)) == set(["((Leaf,(Leaf,Leaf)),Leaf)",
                                      "(Leaf,((Leaf,Leaf),Leaf))",
                                      "((Leaf,Leaf),(Leaf,Leaf))",
                                      "(Leaf,(Leaf,(Leaf,Leaf)))",
                                      "(((Leaf,Leaf),Leaf),Leaf)"
                                  ]))
test_Count_List_Until_Timeout(treeGram, "Tree",10)

#unrank

assert(treeGram["Tree"].unrank(4,1) == "(Leaf,((Leaf,Leaf),Leaf))")
assert(treeGram["Tree"].unrank(3,0) == "(Leaf,(Leaf,Leaf))")
test_unrank(treeGram, "Tree", 8)

#rank

assert(treeGram["Tree"].rank("Leaf") == 0)
assert(treeGram["Tree"].rank("(Leaf,((Leaf,Leaf),Leaf))") == 1)
assert(treeGram["Tree"].rank("(Leaf,(Leaf,Leaf))") == 0)
test_rank(treeGram, "Tree", 6)


Test unrank ok
Right side of union rank
newObj : Leaf
c0: 0
Right side of union rank
newObj : Leaf
c0: 0
Right side of union rank
newObj : Leaf
c0: 0
Right side of union rank
newObj : Leaf
c0: 0
Right side of union rank
newObj : Leaf
c0: 0
Right side of union rank
newObj : Leaf
c0: 0
Right side of union rank
newObj : Leaf
c0: 0
Right side of union rank
newObj : Leaf
c0: 0
Right side of union rank
newObj : Leaf
c0: 0
Right side of union rank
newObj : Leaf
c0: 0
Right side of union rank
newObj : Leaf
c0: 0
Right side of union rank
newObj : Leaf
c0: 0
Right side of union rank
newObj : Leaf
c0: 0
Right side of union rank
newObj : Leaf
c0: 0
Right side of union rank
newObj : Leaf
c0: 0
Right side of union rank
newObj : Leaf
c0: 0
Right side of union rank
newObj : Leaf
c0: 0
Right side of union rank
newObj : Leaf
c0: 0
Right side of union rank
newObj : Leaf
c0: 0
Right side of union rank
newObj : Leaf
c0: 0
Right side of union rank
newObj : Leaf
c0: 0
Right side of union rank
newObj : Leaf
c

__Grammar for Fibonacci's words :__

F --> epsilon | aF | baF | b 

In [122]:
#Grammar definition for Fibonacci words

fiboGram = {"Fib" : UnionRule("Vide", "Cas1",consUnion,revUnion),
            "Cas1" : UnionRule("CasAu", "Cas2",consUnion,revUnion),
            "Cas2" : UnionRule("AtomB", "CasBAu",consUnion,revUnion),
            "Vide" : EpsilonRule(""),
            "CasAu" : ProductRule("AtomA", "Fib",consProd,revProd),
            "AtomA" : SingletonRule("A"),
            "AtomB" : SingletonRule("B"),
            "CasBAu" : ProductRule("AtomB", "CasAu",consProd,revProd)}
init_grammar(fiboGram)

In [123]:
#tests for the fibonacci words grammar

#valuation

assert (fiboGram["Fib"].valuation() == 0)
assert (fiboGram["Cas1"].valuation() == 1)
assert (fiboGram["Cas2"].valuation() == 1)
assert (fiboGram["Vide"].valuation() == 0)
assert (fiboGram["CasAu"].valuation() == 1)
assert (fiboGram["AtomA"].valuation() == 1)
assert (fiboGram["AtomB"].valuation() == 1)
assert (fiboGram["CasBAu"].valuation() == 2)

#count

assert (fiboGram["Fib"].count(0) == 1)
assert (fiboGram["Fib"].count(1) == 2)
assert (fiboGram["Fib"].count(2) == 3)
assert (fiboGram["Fib"].count(3) == 5)
assert (fiboGram["Fib"].count(4) == 8)
assert (fiboGram["Fib"].count(5) == 13)
assert (fiboGram["Fib"].count(6) == 21)
assert (fiboGram["Fib"].count(7) == 34)
assert (fiboGram["Fib"].count(8) == 55)
assert (fiboGram["Fib"].count(9) == 89)

#listR

assert (cleanList(fiboGram["Fib"].listR(0)) == [""])
assert (set(cleanList(fiboGram["Fib"].listR(1))) == set(["A","B"]))
assert (set(cleanList(fiboGram["Fib"].listR(2))) == set(["AA","AB","BA"]))
assert (set(cleanList(fiboGram["Fib"].listR(3))) == set(["AAA","AAB","ABA","BAA","BAB"]))
assert (set(cleanList(fiboGram["Fib"].listR(4))) == set(["AAAA","AAAB","AABA","ABAA","ABAB","BAAA",
                                              "BAAB","BABA"]))

test_Count_List_Until_Timeout(fiboGram, "Fib",10)

#unrank

assert(cleanString(fiboGram["Fib"].unrank(4,1)) == "AAAB")
assert(cleanString(fiboGram["Fib"].unrank(3,3)) == "BAA")
test_unrank(fiboGram, "Fib", 8)

#rank

test_rank(fiboGram,"Fib", 6)
#l = fiboGram["Fib"].listR(1)
#print(l[1])
#fiboGram["Fib"].rank(l[1])

Test unrank ok
Right side of union rank
newObj : L|[A,L|]
c0: 0
Right side of union rank
newObj : R|L|B
c0: 0
Right side of union rank
newObj : L|B
c0: 1
Right side of union rank
newObj : L|[A,R|L|[A,L|]]
c0: 0
Right side of union rank
newObj : L|[A,L|]
c0: 0
Right side of union rank
newObj : L|[A,R|R|L|B]
c0: 0
Right side of union rank
newObj : R|L|B
c0: 0
Right side of union rank
newObj : L|B
c0: 1
Right side of union rank
newObj : R|R|[B,[A,L|]]
c0: 0
Right side of union rank
newObj : R|[B,[A,L|]]
c0: 2
Right side of union rank
newObj : [B,[A,L|]]
c0: 0
Right side of union rank
newObj : L|[A,R|L|[A,R|L|[A,L|]]]
c0: 0
Right side of union rank
newObj : L|[A,R|L|[A,L|]]
c0: 0
Right side of union rank
newObj : L|[A,L|]
c0: 0
Right side of union rank
newObj : L|[A,R|L|[A,R|R|L|B]]
c0: 0
Right side of union rank
newObj : L|[A,R|R|L|B]
c0: 0
Right side of union rank
newObj : R|L|B
c0: 0
Right side of union rank
newObj : L|B
c0: 1
Right side of union rank
newObj : L|[A,R|R|R|[B,[A,L|]]]
c0:

__Grammar for all words formed from {a,b} :__

W --> epsilon | aW | bW

In [124]:
#Grammar definition for all the words with A and B letters
abGram = {"Words" : UnionRule("Vide","Cas1",consUnion,revUnion),
          "Cas1" : UnionRule("CasAu","CasBu",consUnion,revUnion),
          "AtomA" : SingletonRule("A"),
          "AtomB" : SingletonRule("B"),
          "CasAu" : ProductRule("AtomA", "Words",consProd,revProd),
          "CasBu" : ProductRule("AtomB", "Words",consProd,revProd),
          "Vide" : EpsilonRule(""),
          }
init_grammar(abGram) 

In [125]:
# tests for the (a,b) words grammar

#valuation

assert (abGram["Words"].valuation() == 0)
assert (abGram["Cas1"].valuation() == 1)
assert (abGram["AtomA"].valuation() == 1)
assert (abGram["AtomB"].valuation() == 1)
assert (abGram["CasAu"].valuation() == 1)
assert (abGram["CasBu"].valuation() == 1)
assert (abGram["Vide"].valuation() == 0)

#count

assert (abGram["Words"].count(0) == 1)
assert (abGram["Words"].count(1) == 2)
assert (abGram["Words"].count(2) == 4)
assert (abGram["Words"].count(3) == 8)
assert (abGram["Words"].count(4) == 16)
assert (abGram["Words"].count(5) == 32)
assert (abGram["Words"].count(6) == 64)
assert (abGram["Words"].count(7) == 128)

#list

assert (cleanList(abGram["Words"].listR(0)) == [""])
assert (set(cleanList(abGram["Words"].listR(1))) == set(["A","B"]))
assert (set(cleanList(abGram["Words"].listR(2))) == set(["AA", "AB", "BA", "BB"]))
assert (set(cleanList(abGram["Words"].listR(3))) == set(["AAA", "AAB", "ABA", "ABB",
                                              "BAA", "BAB", "BBA", "BBB"]))
test_Count_List_Until_Timeout(abGram, "Words",10)

#unrank

assert(cleanString(abGram["Words"].unrank(3,7)) == "BBB")
assert(cleanString(abGram["Words"].unrank(5,0)) == "AAAAA")
test_unrank(abGram, "Words", 8)

#rank

test_rank(abGram,"Words", 6)

Test unrank ok
Right side of union rank
newObj : L|[A,L|]
c0: 0
Right side of union rank
newObj : R|[B,L|]
c0: 0
Right side of union rank
newObj : [B,L|]
c0: 1
Right side of union rank
newObj : L|[A,R|L|[A,L|]]
c0: 0
Right side of union rank
newObj : L|[A,L|]
c0: 0
Right side of union rank
newObj : L|[A,R|R|[B,L|]]
c0: 0
Right side of union rank
newObj : R|[B,L|]
c0: 0
Right side of union rank
newObj : [B,L|]
c0: 1
Right side of union rank
newObj : R|[B,R|L|[A,L|]]
c0: 0
Right side of union rank
newObj : [B,R|L|[A,L|]]
c0: 2
Right side of union rank
newObj : L|[A,L|]
c0: 0
Right side of union rank
newObj : R|[B,R|R|[B,L|]]
c0: 0
Right side of union rank
newObj : [B,R|R|[B,L|]]
c0: 2
Right side of union rank
newObj : R|[B,L|]
c0: 0
Right side of union rank
newObj : [B,L|]
c0: 1
Right side of union rank
newObj : L|[A,R|L|[A,R|L|[A,L|]]]
c0: 0
Right side of union rank
newObj : L|[A,R|L|[A,L|]]
c0: 0
Right side of union rank
newObj : L|[A,L|]
c0: 0
Right side of union rank
newObj : L|[A,R|

newObj : R|[B,R|R|[B,R|L|[A,R|L|[A,L|]]]]
c0: 0
Right side of union rank
newObj : [B,R|R|[B,R|L|[A,R|L|[A,L|]]]]
c0: 8
Right side of union rank
newObj : R|[B,R|L|[A,R|L|[A,L|]]]
c0: 0
Right side of union rank
newObj : [B,R|L|[A,R|L|[A,L|]]]
c0: 4
Right side of union rank
newObj : L|[A,R|L|[A,L|]]
c0: 0
Right side of union rank
newObj : L|[A,L|]
c0: 0
Right side of union rank
newObj : R|[B,R|R|[B,R|R|[B,R|L|[A,R|R|[B,L|]]]]]
c0: 0
Right side of union rank
newObj : [B,R|R|[B,R|R|[B,R|L|[A,R|R|[B,L|]]]]]
c0: 16
Right side of union rank
newObj : R|[B,R|R|[B,R|L|[A,R|R|[B,L|]]]]
c0: 0
Right side of union rank
newObj : [B,R|R|[B,R|L|[A,R|R|[B,L|]]]]
c0: 8
Right side of union rank
newObj : R|[B,R|L|[A,R|R|[B,L|]]]
c0: 0
Right side of union rank
newObj : [B,R|L|[A,R|R|[B,L|]]]
c0: 4
Right side of union rank
newObj : L|[A,R|R|[B,L|]]
c0: 0
Right side of union rank
newObj : R|[B,L|]
c0: 0
Right side of union rank
newObj : [B,L|]
c0: 1
Right side of union rank
newObj : R|[B,R|R|[B,R|R|[B,R|R|[B,R

__Grammar for Dyck's words :__

D --> epsilon | PD <br>
P --> (D)
    

In [135]:
#Grammar definition for Dyck words
dyckGram = {"Dyck" : UnionRule("Vide","Cas1",consUnion,revUnion),
            "Cas1" : ProductRule("Parenthesis","Dyck",consProd,revProd),
            "Parenthesis" : ProductRule("AtomG","CasD",consProd,revProd),
            "CasD" : ProductRule("Dyck","AtomD",consProd,revProd),
            "Vide" : EpsilonRule(""),
            "AtomG" : SingletonRule("("),
            "AtomD" : SingletonRule(")")
            }
init_grammar(dyckGram)

In [136]:
# tests for the Dyck words grammar

#valuation

assert (dyckGram["Dyck"].valuation() == 0)
assert (dyckGram["Cas1"].valuation() == 2)
assert (dyckGram["Vide"].valuation() == 0)
assert (dyckGram["AtomG"].valuation() == 1)
assert (dyckGram["AtomD"].valuation() == 1)
assert (dyckGram["Parenthesis"].valuation() == 2)
assert (dyckGram["CasD"].valuation() == 1)

#count

assert (dyckGram["Dyck"].count(0) == 1)
assert (dyckGram["Dyck"].count(1) == 0)
assert (dyckGram["Dyck"].count(2) == 1)
assert (dyckGram["Dyck"].count(3) == 0)
assert (dyckGram["Dyck"].count(4) == 2)
assert (dyckGram["Dyck"].count(5) == 0)
assert (dyckGram["Dyck"].count(6) == 5)
assert (dyckGram["Dyck"].count(7) == 0)
assert (dyckGram["Dyck"].count(8) == 14)
assert (dyckGram["Dyck"].count(9) == 0)

#list
assert (cleanList(dyckGram["Dyck"].listR(0)) == [""])
assert (set(cleanList(dyckGram["Dyck"].listR(1))) == set([]))
assert (set(cleanList(dyckGram["Dyck"].listR(2))) == set(["()"]))
assert (set(cleanList(dyckGram["Dyck"].listR(4))) == set(["()()","(())"]))
assert (set(cleanList(dyckGram["Dyck"].listR(6))) == set(["()()()","()(())","(())()",
                                             "(()())","((()))"]))
test_Count_List_Until_Timeout(dyckGram, "Dyck",10)

#unrank

assert(cleanString(dyckGram["Dyck"].unrank(2,0)) == "()")
assert(cleanString(dyckGram["Dyck"].unrank(6,2)) == "(())()")
test_unrank(dyckGram, "Dyck", 8)

#rank

#test_rank(dyckGram,"Dyck", 6)
l = dyckGram["Dyck"].listR(2)
dyckGram["Dyck"].rank(l[0])

Test unrank ok
Right side of union rank
newObj : [[(,[L|,)]],L|]
c0: 0
obj= [[(,[L|,)]],L|]
k=3 l=1
[(,[L|,)]]
L|
obj= [(,[L|,)]]
k=1 l=2
(
[L|,)]
obj= [L|,)]
k=1 l=1
L|
)


1

__Grammar for all the words without 3 times the same letter in a row, with {a,b} :__

W  --> espilon | aN1 | bN2 <br>
N1  --> bN2 | aAB | epsilon <br>
N2  --> aN1 | bBA | epsilon <br>
AB --> bN2 | epsilon <br>
BA --> aN1 | epsilon <br>

In [96]:
#Grammar definition for the (A,B) words without 3 times the same letter in a row
ab3Gram = {"Words" : UnionRule("Vide","Cas1",consUnion,revUnion),
              "Cas1" : UnionRule("CasA","CasB",consUnion,revUnion),
              "CasA" : ProductRule("AtomA","CasA1",consProd,revProd),
              "CasA1" : UnionRule("CasB","CasA2",consUnion,revUnion),
              "CasA2" : UnionRule("CasAB","Vide",consUnion,revUnion),
              "CasB" : ProductRule("AtomB","CasB1",consProd,revProd),
              "CasB1" : UnionRule("CasA","CasB2",consUnion,revUnion),
              "CasB2" : UnionRule("CasBA","Vide",consUnion,revUnion),
              "CasAB" : ProductRule("AtomA","CasAB1",consProd,revProd),
              "CasAB1" : UnionRule("CasB","Vide",consUnion,revUnion),
              "CasBA" : ProductRule("AtomB","CasBA1",consProd,revProd),
              "CasBA1" : UnionRule("CasA","Vide",consUnion,revUnion),
              "Vide" : EpsilonRule(""),
              "AtomA" : SingletonRule("A"),
              "AtomB" : SingletonRule("B")
             }

init_grammar(ab3Gram)

In [97]:
# tests for the (A,B) words without 3 times the same letter in a row grammar

#valuation
assert (ab3Gram["Words"].valuation() == 0)
assert (ab3Gram["Cas1"].valuation() == 1)
assert (ab3Gram["CasA"].valuation() == 1)
assert (ab3Gram["CasA1"].valuation() == 0)
assert (ab3Gram["CasA2"].valuation() == 0)
assert (ab3Gram["CasB"].valuation() == 1)
assert (ab3Gram["CasB1"].valuation() == 0)
assert (ab3Gram["CasB2"].valuation() == 0)
assert (ab3Gram["CasAB"].valuation() == 1)
assert (ab3Gram["CasAB1"].valuation() == 0)
assert (ab3Gram["CasBA"].valuation() == 1)
assert (ab3Gram["CasBA1"].valuation() == 0)
assert (ab3Gram["Vide"].valuation() == 0)
assert (ab3Gram["AtomA"].valuation() == 1)

#count

assert (ab3Gram["Words"].count(0) == 1)
assert (ab3Gram["Words"].count(1) == 2)
assert (ab3Gram["Words"].count(2) == 4)
assert (ab3Gram["Words"].count(3) == 6)
assert (ab3Gram["Words"].count(4) == 10)

#list
assert (cleanList(ab3Gram["Words"].listR(0)) == [""])
assert (set(cleanList(ab3Gram["Words"].listR(1))) == set(["A","B"]))
assert (set(cleanList(ab3Gram["Words"].listR(2))) == set(["AA","AB","BA","BB"]))
assert (set(cleanList(ab3Gram["Words"].listR(3))) == set(["AAB","ABA","ABB","BAA","BAB","BBA"]))
assert (set(cleanList(ab3Gram["Words"].listR(4))) == set(["AABA","AABB","ABAA","ABAB","ABBA",
                                              "BAAB","BABA","BABB","BBAA","BBAB"]))
test_Count_List_Until_Timeout(ab3Gram, "Words",10)

#unrank
assert(cleanString(ab3Gram["Words"].unrank(2,2)) == "BA")
assert(cleanString(ab3Gram["Words"].unrank(4,5)) == "BABA")
test_unrank(ab3Gram, "Words", 8)

#rank

test_rank(ab3Gram,"Words", 6)

Test unrank ok
Test rank ok


__Grammar for all the words{A,B} where Card(A) = Card(B) :__

W --> epsilon | aBW | bAW <br>
B --> b | aBB <br>
A --> a | bAA 

In [98]:
#Grammar of all the words{A,B} where Card(A) = Card(B)
GramABequal = { "Words"     : UnionRule("Vide", "Cas0",consUnion,revUnion),
                "Cas0"      : ProductRule("Cas1","Words",consProd,revProd),
                "Cas1"      : UnionRule("CasaB","CasbA",consUnion,revUnion),
                "CasaB"     : ProductRule("AtomA","CasB",consProd,revProd),
                "CasB"      : UnionRule("AtomB","CasaBB",consUnion,revUnion),
                "CasaBB"    : ProductRule("AtomA","CasBB",consProd,revProd),
                "CasBB"     : ProductRule("CasB","CasB",consProd,revProd),
                "CasbA"     : ProductRule("AtomB","CasA",consProd,revProd),
                "CasA"      : UnionRule("AtomA","CasbAA",consUnion,revUnion),
                "CasbAA"    : ProductRule("AtomB", "CasAA",consProd,revProd),
                "CasAA"     : ProductRule("CasA","CasA",consProd,revProd),
                "AtomA"     : SingletonRule("A"),
                "AtomB"     : SingletonRule("B"),
                "Vide"      : EpsilonRule("")}

init_grammar(GramABequal)

In [99]:
# tests for the (a,b) words where Card(A) = Card(B) grammar

#valuation

assert (GramABequal["Words"].valuation() == 0)
assert (GramABequal["Cas0"].valuation() == 2)
assert (GramABequal["Cas1"].valuation() == 2)
assert (GramABequal["CasaB"].valuation() == 2)
assert (GramABequal["CasbA"].valuation() == 2)
assert (GramABequal["CasA"].valuation() == 1)
assert (GramABequal["CasB"].valuation() == 1)
assert (GramABequal["CasbAA"].valuation() == 3)
assert (GramABequal["CasAA"].valuation() == 2)
assert (GramABequal["CasaBB"].valuation() == 3)
assert (GramABequal["CasBB"].valuation() == 2)
assert (GramABequal["AtomA"].valuation() == 1)
assert (GramABequal["AtomB"].valuation() == 1)
assert (GramABequal["Vide"].valuation() == 0)

#count 

assert (GramABequal["Words"].count(0) == 1)
assert (GramABequal["Words"].count(1) == 0)
assert (GramABequal["Words"].count(2) == 2)
assert (GramABequal["Words"].count(3) == 0)
assert (GramABequal["Words"].count(4) == 6)
assert (GramABequal["Words"].count(5) == 0)
assert (GramABequal["Words"].count(6) == 20)

#list

assert (cleanList(GramABequal["Words"].listR(0)) == [""])
assert (set(cleanList(GramABequal["Words"].listR(1))) == set([]))
assert (set(cleanList(GramABequal["Words"].listR(2))) == set(["AB","BA"]))
assert (set(cleanList(GramABequal["Words"].listR(4))) == set(["AABB","ABAB","BBAA",
                                                     "BABA","ABBA","BAAB"]))
test_Count_List_Until_Timeout(GramABequal, "Words",10)

#unrank

assert(cleanString(GramABequal["Words"].unrank(2,1)) == "BA")
assert(cleanString(GramABequal["Words"].unrank(4,5)) == "BBAA")
test_unrank(GramABequal, "Words", 8)

#rank

test_rank(GramABequal,"Words", 6)

Test unrank ok
(n,i) :
(2, 1)
rank result :
0
corresponding element :
1
entire list, and matched element
['R|[L|[A,L|B],L|]', 'R|[R|[B,L|A],L|]']
R|[R|[B,L|A],L|]


AssertionError: 

__Grammar for all the Palyndromes on {a,b,c} :__

W  --> epsilon | W | a | b | c | aWa | bWb | cWc

In [100]:
#Grammar of Palyndromes{A,B,C}
PalindromeABC = {   "Words"     : UnionRule("Vide", "Mot",consUnion,revUnion),
                    "Mot"       : UnionRule("ABC","ABCmotABC",consUnion,revUnion),
                    "ABCmotABC" : UnionRule("AmotA", "BCmotBC",consUnion,revUnion),
                    "BCmotBC"   : UnionRule("BmotB","CmotC",consUnion,revUnion),
                    "AmotA"     : ProductRule("AtomA","MotA",consProd,revProd),
                    "MotA"      : ProductRule("Words","AtomA",consProd,revProd),
                    "BmotB"     : ProductRule("AtomB","MotB",consProd,revProd),
                    "MotB"      : ProductRule("Words","AtomB",consProd,revProd),
                    "CmotC"     : ProductRule("AtomC","MotC",consProd,revProd),
                    "MotC"      : ProductRule("Words","AtomC",consProd,revProd),
                    "ABC"       : UnionRule("AtomA","BC",consUnion,revUnion),
                    "BC"        : UnionRule("AtomB","AtomC",consUnion,revUnion),
                    "AtomA"     : SingletonRule("A"),
                    "AtomB"     : SingletonRule("B"),
                    "AtomC"     : SingletonRule("C"),
                    "Vide"      : EpsilonRule("")}

init_grammar(PalindromeABC)

In [101]:
# tests for the Palyndromes{A,B,C} grammar

#valuation

assert (PalindromeABC["Words"].valuation() == 0)
assert (PalindromeABC["Mot"].valuation() == 1)
assert (PalindromeABC["ABCmotABC"].valuation() == 2)
assert (PalindromeABC["BCmotBC"].valuation() == 2)
assert (PalindromeABC["AmotA"].valuation() == 2)
assert (PalindromeABC["BmotB"].valuation() == 2)
assert (PalindromeABC["CmotC"].valuation() == 2)
assert (PalindromeABC["MotA"].valuation() == 1)
assert (PalindromeABC["MotB"].valuation() == 1)
assert (PalindromeABC["MotC"].valuation() == 1)
assert (PalindromeABC["ABC"].valuation() == 1)
assert (PalindromeABC["BC"].valuation() == 1)
assert (PalindromeABC["AtomA"].valuation() == 1)
assert (PalindromeABC["AtomB"].valuation() == 1)
assert (PalindromeABC["AtomC"].valuation() == 1)
assert (PalindromeABC["Vide"].valuation() == 0)

#count n+1 = (n-1)³

assert (PalindromeABC["Words"].count(0) == 1)
assert (PalindromeABC["Words"].count(1) == 3)
assert (PalindromeABC["Words"].count(2) == 3)
assert (PalindromeABC["Words"].count(3) == 9)
assert (PalindromeABC["Words"].count(4) == 9)
assert (PalindromeABC["Words"].count(5) == 27)
assert (PalindromeABC["Words"].count(6) == 27)

#list

assert (cleanList(PalindromeABC["Words"].listR(0)) == [""])
assert (set(cleanList(PalindromeABC["Words"].listR(1))) == set(["A","B","C"]))
assert (set(cleanList(PalindromeABC["Words"].listR(2))) == set(["AA","BB","CC"]))
assert (set(cleanList(PalindromeABC["Words"].listR(3))) == set(["AAA","ABA","ACA",
                                                    "BAB","BBB","BCB",
                                                    "CAC","CBC","CCC"]))
assert (set(cleanList(PalindromeABC["Words"].listR(4))) == set(["AAAA","ABBA","ACCA",
                                                    "BAAB","BBBB","BCCB",
                                                    "CAAC","CBBC","CCCC"]))
test_Count_List_Until_Timeout(PalindromeABC, "Words",10)

#unrank

assert(cleanString(PalindromeABC["Words"].unrank(2,1)) == "BB")
assert(cleanString(PalindromeABC["Words"].unrank(4,5)) == "BCCB")
test_unrank(PalindromeABC, "Words", 8)

#rank

test_rank(PalindromeABC,"Words", 6)

Test unrank ok
Test rank ok
